In [1]:
from tensorflow import keras
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Conv1D, Conv1DTranspose, Dense
from tensorflow.keras.constraints import max_norm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

2022-11-08 13:19:50.790380: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-08 13:19:50.855879: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-08 13:19:50.858626: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-08 13:19:50.858634: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not ha

Load data

In [2]:
X_train = np.load('../Data/X_train_C.npy')
Y_train = np.load('../Data/Y_train_C.npy')

X_test = np.load('../Data/X_val_C.npy')
Y_test = np.load('../Data/Y_val_C.npy')

Normalize inputs

In [3]:
Level = np.min(X_train)
Height = np.max(X_train) - np.min(X_train)

X_train = (X_train - Level) / Height
X_test = (X_test - Level) / Height

Reshape data

In [4]:
X_train = X_train[:, :, np.newaxis]
X_test = X_test[:, :, np.newaxis]

Callbacks

In [5]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=500, monitor='val_loss', restore_best_weights=True) # Stop if the validation loss is not improving
learning_rate_cb = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=100) # Decrease the learning rate if the validation loss is not improving
cb_list = [early_stopping_cb, learning_rate_cb]

Initialize model

In [6]:
max_norm_value = 2.0

model = Sequential()
model.add(Conv1D(64, kernel_size=3, kernel_constraint=max_norm(max_norm_value), activation='relu', kernel_initializer='he_uniform', input_shape=X_train[0].shape))
model.add(Conv1D(32, kernel_size=3, kernel_constraint=max_norm(max_norm_value), activation='relu', kernel_initializer='he_uniform'))
model.add(Conv1D(16, kernel_size=3, kernel_constraint=max_norm(max_norm_value), activation='relu', kernel_initializer='he_uniform'))
model.add(Conv1DTranspose(16, kernel_size=3, kernel_constraint=max_norm(max_norm_value), activation='relu', kernel_initializer='he_uniform'))
model.add(Conv1DTranspose(32, kernel_size=3, kernel_constraint=max_norm(max_norm_value), activation='relu', kernel_initializer='he_uniform'))
model.add(Conv1DTranspose(64, kernel_size=3, kernel_constraint=max_norm(max_norm_value), activation='relu', kernel_initializer='he_uniform'))
model.add(Conv1D(320, kernel_size=3, kernel_constraint=max_norm(max_norm_value), activation='relu', padding='same'))
model.add(Conv1D(1, kernel_size=3, kernel_constraint=max_norm(max_norm_value), activation='tanh', padding='same'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["mae"])

2022-11-08 13:19:56.778194: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-11-08 13:19:56.778226: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: tretiako-Stealth-GS77-12UGS
2022-11-08 13:19:56.778229: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: tretiako-Stealth-GS77-12UGS
2022-11-08 13:19:56.778348: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 515.65.1
2022-11-08 13:19:56.778369: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: NOT_FOUND: could not find kernel module information in driver version file contents: "NVRM version: NVIDIA UNIX Open Kernel Module for x86_64  515.65.01  Release Build  (dvs-builder@U16-T11-05-2)  Wed Jul 20 13:54:56 UTC 2022
GCC version:  gcc version 11.3.0 (Ubuntu 11.3.0-1ubuntu1~22.04) 
"
2022-11-08 13:19

Training

In [7]:
noise_level = 0.01
X_test_N = X_test + np.random.normal(0, noise_level, X_test.shape)
X_train_N = X_train + np.random.normal(0, noise_level, X_train.shape)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=["mae"])
history = model.fit(X_train_N, X_train, epochs=2000, validation_data=(X_test_N, X_test), batch_size=32, callbacks=cb_list, verbose=False)

KeyboardInterrupt: 

Plot training history

In [ ]:
h = history.history

training_data = pd.DataFrame({"training loss": h["loss"], "test_loss": h["val_loss"]})
training_data.index.name = 'CNN Epoch'

fig = training_data.plot()
fig.set_xlabel(r'epoch_count', fontsize=12)
fig.set_ylabel(r'Loss', fontsize=12)
fig.set_ylim(0,0.1)

Visualize denoiser performance

In [ ]:
test = model.predict(X_test_N)  # Predicted output

index = 200

fig, ax = plt.subplots()
ax.plot(np.linspace(-4,4,320), X_test_N[index], label='noisy input')
ax.plot(np.linspace(-4,4,320), X_test[index], label='original input', ls='--')
ax.plot(np.linspace(-4,4,320), test[index], label='output', ls='--')
ax.legend(loc=0)
ax.set_xlabel(r'$\Delta_{\mu}/\omega_{L}$', fontsize=12)
ax.set_ylabel('DR signal', fontsize=12)

fig, ax = plt.subplots()
ax.plot(np.linspace(-4,4,320), X_test_N[index] - X_test[index], label='diference')
ax.legend(loc=0)
ax.set_ylim(-0.5, 0.5)
ax.set_xlabel(r'$\Delta_{\mu}/\omega_{L}$', fontsize=12)
ax.set_ylabel('Predicted - Expected', fontsize=12)

Plot model

In [ ]:
from keras.utils import plot_model
plot_model(model, show_shapes=True, to_file='model.png',show_layer_activations=True)

### Conclusions

Looks good, but when I try to integrate it with transfer learning, it does not help. Needs further investigation.